In [ ]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN


In [2]:
data_minhashing=pd.read_csv("/Users/sararedaelli/Desktop/AMAZON REVIEWS/minhashing_results.csv")
df=data_minhashing
df.shape

(2968263, 2)

In [3]:
df.columns

Index(['signature', 'rating'], dtype='object')

## CLUSTER

# Each cell contains an alternative way to do clustering

In [ ]:
#KMEANS with minibatch (FASTER THAN NORMAL K-MEANS)
# --- 1. Prepare data ---
# Assuming df['signature'] contains lists or arrays of length 100
X = np.array(df['signature'].tolist())
y = df['rating'].values  # true labels (optional, per valutazione)

# --- 2. Run Mini-Batch K-Means ---
mini_kmeans = MiniBatchKMeans(
    n_clusters=5,
    random_state=42,
    batch_size=100,     # <--- puoi adattare la dimensione del batch
    n_init='auto',      # gestito automaticamente in sklearn >=1.4
    max_iter=1000       # più iterazioni, ma su batch piccoli
)

clusters = mini_kmeans.fit_predict(X)


: 

In [ ]:
#K-MEANS  
# --- 1. Prepare data ---
# Assuming df['signature'] contains lists or arrays of length 100
X = np.array(df['signature'].tolist())
y = df['score'].values  # true labels

# --- 2. Run K-Means ---
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)


NameError: name 'np' is not defined

In [ ]:
#K-MEANS after PCA  (reduce dimensionality before doing clustering)
# --- 1. Prepare data ---
# Assuming df['signature'] contains lists or arrays of length 100
X = np.array(df['signature'].tolist())

# --- 2. PCA: keep enough components to explain 90% of variance ---
pca = PCA(n_components=0.9, random_state=42)
X_pca = pca.fit_transform(X)

print(f"Number of PCA components selected: {pca.n_components_}")
print(f"Cumulative explained variance: {np.sum(pca.explained_variance_ratio_):.2%}")

# --- 3. Run K-Means on PCA-reduced data ---
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_pca)

In [ ]:
#DBSCAN AFTER PCA
# --- 1. Prepare data ---
X = np.array(df['signature'].tolist())

# --- 2. PCA: keep 90% cumulative variance ---
pca = PCA(n_components=0.9, random_state=42)
X_pca = pca.fit_transform(X)

print(f"Number of PCA components selected: {pca.n_components_}")
print(f"Cumulative explained variance: {np.sum(pca.explained_variance_ratio_):.2%}")

# --- 3. Estimate a good eps using the k-distance plot ---
min_samples = 3
neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors_fit = neighbors.fit(X_pca)
distances, indices = neighbors_fit.kneighbors(X_pca)

# Sort distances for the k-distance graph
distances = np.sort(distances[:, -1])

plt.figure(figsize=(8,5))
plt.plot(distances)
plt.title("k-distance graph (use the 'elbow' to choose eps)")
plt.xlabel("Points sorted by distance")
plt.ylabel(f"{min_samples}-NN distance")
plt.grid(True)
plt.show()

# 👉 Look at the plot and choose an eps value where the curve sharply increases.
# For example, eps = 0.5 (you’ll set it after visual inspection)
eps = 0.5  # <-- adjust this after checking the plot visually

# --- 4. Run DBSCAN ---
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
clusters = dbscan.fit_predict(X_pca)

In [ ]:
#DB-SCAN
# --- 1. Prepare data ---
# Assuming df['signature'] contains lists or arrays of length 100
X = np.array(df['signature'].tolist())

# --- 2. Estimate a good eps using the k-distance plot ---
min_samples = 3
neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors_fit = neighbors.fit(X)
distances, indices = neighbors_fit.kneighbors(X)

# Sort distances for the k-distance graph
distances = np.sort(distances[:, -1])

plt.figure(figsize=(8,5))
plt.plot(distances)
plt.title("k-distance graph (use the 'elbow' to choose eps)")
plt.xlabel("Points sorted by distance")
plt.ylabel(f"{min_samples}-NN distance")
plt.grid(True)
plt.show()

# 👉 Inspect the plot and choose an eps value around the 'elbow'
eps = 0.5  # <-- adjust after checking the curve

# --- 3. Run DBSCAN on the original data (no PCA) ---
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
clusters = dbscan.fit_predict(X)


## EVALUATION

In [ ]:
#EVALUATION using DAVIES-BOULDIN INDEX
from sklearn.metrics import davies_bouldin_score
# --- 3. Evaluate with Davies-Bouldin Index ---
db = davies_bouldin_score(X, clusters)
print(f"Davies-Bouldin Index (DB): {db:.4f}")

In [ ]:
#PLOT in two dimensions using PCA and color by cluster
#prepare data 
X = np.array(df['signature'].tolist())  # shape (n_samples, 100)
# --- 4. Optional: visualize clusters ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='tab10', alpha=0.7)
plt.title("K-Means clustering of MinHash signatures")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.colorbar(label="Cluster ID")
plt.show()